In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
BASE_DIR = Path().absolute().parent.parent
sys.path.append(str(BASE_DIR))
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3,Case3T
import numpy as np
import pandas as pd

In [3]:
DIR = BASE_DIR/'scripts_res'/'Case3THyper3'
DIR

WindowsPath('d:/OneDrive/Current/2_Codes/BO_Related/ASBO/ASBO/scripts_res/Case3THyper3')

In [4]:
sol = 'ASBO'
case_name = 'Case3T'
runs_number = 5
n_iter = 200
group = 2
tt = pd.read_csv(DIR/'LHS_Case3T.csv')
hyper_p_list = tt.loc[group*5:4+group*5,:]
hyper_p_list

,0,1,2,3
10,4.0,2.5,0.80,0.95
11,5.0,2.0,0.76,0.98
12,4.0,2.5,0.78,0.98
13,3.0,1.5,0.78,0.98
14,3.0,2.5,0.80,0.92


In [5]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

obj_list = []
x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []
all_res_list = []

def func(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3T(x)
    cons = [c1,c2,c3,c4]
    return -obj,cons

nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)
for i in range(5*group,5+group*5,1):
    hyper_p = hyper_p_list.loc[i,:]

    m1,m2,p1,p2 = hyper_p[0],hyper_p[1],hyper_p[2],hyper_p[3]
    print(m1,m2,p1,p2)
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        try:
            optimizer.maximize(init_points=100, n_iter=n_iter,p_hyper=[[m1,m2,p1,p2,1e-5]],
                                acquisition_function=acquisition_function)
            print(optimizer.max)
            x_1_list.extend([optimizer.max['params']['x1']])
            x_2_list.extend([optimizer.max['params']['x2']])
            x_3_list.extend([optimizer.max['params']['x3']])
            x_4_list.extend([optimizer.max['params']['x4']])
            x_5_list.extend([optimizer.max['params']['x5']])  
            obj_list.extend([optimizer.max['target']])                                  
            con_1_list.extend([optimizer.max['constraint'][0]])
            con_2_list.extend([optimizer.max['constraint'][1]])
            con_3_list.extend([optimizer.max['constraint'][2]])
            con_4_list.extend([optimizer.max['constraint'][3]])        
            all_res = [item['target'] for item in optimizer.res]
            all_fea = [item['allowed'] for item in optimizer.res]
            modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
            all_res_list.append(modified_res)        
        except Exception as e:
            print(e)
            obj_list.append(None)
            x_1_list.append(None)
            x_2_list.append(None)
            x_3_list.append(None)
            x_4_list.append(None)
            x_5_list.append(None)                        
            con_1_list.append(None)
            con_2_list.append(None)
            con_3_list.append(None)
            con_4_list.append(None) 
            modified_res = [-np.inf for _ in range(100+n_iter)]
            all_res_list.append(modified_res)       

        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'con_4': con_4_list
        })
        df2 = pd.DataFrame(all_res_list, columns=[list(range(100+n_iter))])
        df.to_csv(DIR/f'{sol}_Hyper_Results_{case_name}_{runs_number}_{group}.csv', index=False)
        df2.to_csv(DIR/f'{sol}_Hyper_Results2_{case_name}_{runs_number}_{group}.csv', index=False)

4.0 2.5 0.8 0.95
{'target': -4.645147596721873, 'params': {'x1': 4.017921124847652, 'x2': -0.0004919114042135031, 'x3': -3.990137845904457, 'x4': 1.9901511866175838, 'x5': -6.021870603513328}, 'constraint': array([ 0.52959539, -0.98421911,  1.01911664,  1.1306437 ])}
{'target': 3.51851075401014, 'params': {'x1': 1.998894915973873, 'x2': -0.9988930357601566, 'x3': 0.02531790417115409, 'x4': -0.9921415351564874, 'x5': 0.011068650593991166}, 'constraint': array([-0.73861501, -0.68636444,  2.76019124,  0.5509684 ])}
{'target': -6.054297694431838, 'params': {'x1': 6.038008538880837, 'x2': -5.042309674549407, 'x3': 2.033038448649281, 'x4': 2.9780835955868117, 'x5': -4.035090907179524}, 'constraint': array([-0.94435511, -0.93654414, -0.00990103,  1.45736561])}
{'target': 1.9918453043308797, 'params': {'x1': -0.9499468496756337, 'x2': 1.0778252348552837, 'x3': 0.061861959234842674, 'x4': 0.0021320227339852904, 'x5': 1.1280291929692827}, 'constraint': array([1.80180554, 2.0885756 , 2.9947819 , 